# W_GL_ACCOUNT_D ETL Process
### Staging to Dimension - SCD Type 2 with Effective Dating
**ODI Package:** SILOS_SIL_GLACCOUNTDIMENSION  
**IKM:** BIAPPS Oracle Slowly Changing Dimension

In [ ]:
%sql
-- Step 1: Variable IS_INCREMENTAL - Check if incremental load is enabled
CREATE OR REPLACE TEMP VIEW VAR_IS_INCREMENTAL AS
SELECT 
  CASE 
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL
FROM PRXBI_DW.W_ETL_LOAD_DATES
WHERE PACKAGE_NAME = 'SILOS_SIL_GLACCOUNTDIMENSION'
  AND (DATASOURCE_NUM_ID = 380 OR DATASOURCE_NUM_ID = 380)
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1'

In [ ]:
%sql
-- Step 2: Create Flow Table I$_2625250_1 with NOT EXISTS change detection
DROP VIEW IF EXISTS I$_2625250_1;

CREATE OR REPLACE TEMP VIEW I$_2625250_1 AS
SELECT
  INLINE_VIEW.*,
  COALESCE(INLINE_VIEW.SRC_EFF_FROM_DT, to_date('1900-01-01', 'yyyy-MM-dd')) AS EFFECTIVE_FROM_DT,
  to_date('3714-01-01', 'yyyy-MM-dd') AS EFFECTIVE_TO_DT,
  'I' AS IND_UPDATE
FROM (
  SELECT
    MAX(CHART_OF_ACCOUNTS) AS CHART_OF_ACCOUNTS,
    MAX(CONCATENATED_SEGMENT_VALUE) AS CONCATENATED_SEGMENT_VALUE,
    MAX(BALANCING_SEGMENT_NUM) AS BALANCING_SEGMENT_NUM,
    MAX(BALANCING_SEGMENT_ATTRIB) AS BALANCING_SEGMENT_ATTRIB,
    MAX(COST_CENTER_NUM) AS COST_CENTER_NUM,
    MAX(COST_CENTER_ATTRIB) AS COST_CENTER_ATTRIB,
    MAX(NATURAL_ACCOUNT_NUM) AS NATURAL_ACCOUNT_NUM,
    MAX(NATURAL_ACCOUNT_ATTRIB) AS NATURAL_ACCOUNT_ATTRIB,
    MAX(RECON_TYPE_CODE) AS RECON_TYPE_CODE,
    MAX(ACTIVE_FLG) AS ACTIVE_FLG,
    MAX(CREATED_BY_ID) AS CREATED_BY_ID,
    MAX(CHANGED_BY_ID) AS CHANGED_BY_ID,
    MAX(CREATED_ON_DT) AS CREATED_ON_DT,
    MAX(CHANGED_ON_DT) AS CHANGED_ON_DT,
    MAX(AUX1_CHANGED_ON_DT) AS AUX1_CHANGED_ON_DT,
    MAX(AUX2_CHANGED_ON_DT) AS AUX2_CHANGED_ON_DT,
    MAX(AUX3_CHANGED_ON_DT) AS AUX3_CHANGED_ON_DT,
    MAX(DELETE_FLG) AS DELETE_FLG,
    SRC_EFF_FROM_DT,
    DATASOURCE_NUM_ID,
    INTEGRATION_ID,
    MAX(TENANT_ID) AS TENANT_ID,
    MAX(X_CUSTOM) AS X_CUSTOM
  FROM PRXBI_DW.W_GL_ACCOUNT_DS
  GROUP BY SRC_EFF_FROM_DT, DATASOURCE_NUM_ID, INTEGRATION_ID
) INLINE_VIEW
WHERE NOT EXISTS (
  SELECT 1
  FROM PRXBI_DW.W_GL_ACCOUNT_D T
  WHERE T.SRC_EFF_FROM_DT = INLINE_VIEW.SRC_EFF_FROM_DT
    AND T.DATASOURCE_NUM_ID = INLINE_VIEW.DATASOURCE_NUM_ID
    AND T.INTEGRATION_ID = INLINE_VIEW.INTEGRATION_ID
    AND (CASE WHEN T.CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE T.CHANGED_ON_DT END = 
         CASE WHEN INLINE_VIEW.CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE INLINE_VIEW.CHANGED_ON_DT END)
    AND (CASE WHEN T.AUX1_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE T.AUX1_CHANGED_ON_DT END = 
         CASE WHEN INLINE_VIEW.AUX1_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE INLINE_VIEW.AUX1_CHANGED_ON_DT END)
    AND (CASE WHEN T.AUX2_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE T.AUX2_CHANGED_ON_DT END = 
         CASE WHEN INLINE_VIEW.AUX2_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE INLINE_VIEW.AUX2_CHANGED_ON_DT END)
    AND (CASE WHEN T.AUX3_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE T.AUX3_CHANGED_ON_DT END = 
         CASE WHEN INLINE_VIEW.AUX3_CHANGED_ON_DT IS NULL THEN to_date('3714-01-01', 'yyyy-MM-dd') ELSE INLINE_VIEW.AUX3_CHANGED_ON_DT END)
)

In [ ]:
%sql
-- Step 3: Insert SCD1 table entries (W_GL_ACCOUNT_T1_D) for new natural keys
INSERT INTO PRXBI_DW.W_GL_ACCOUNT_T1_D
(
  SCD1_WID,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  TENANT_ID
)
SELECT
  row_number() OVER (ORDER BY I.DATASOURCE_NUM_ID, I.INTEGRATION_ID) + 
  COALESCE((SELECT MAX(SCD1_WID) FROM PRXBI_DW.W_GL_ACCOUNT_T1_D), 0) AS SCD1_WID,
  I.DATASOURCE_NUM_ID,
  I.INTEGRATION_ID,
  I.TENANT_ID
FROM (
  SELECT DISTINCT
    DATASOURCE_NUM_ID,
    INTEGRATION_ID,
    TENANT_ID
  FROM I$_2625250_1
) I
LEFT OUTER JOIN PRXBI_DW.W_GL_ACCOUNT_T1_D T1
  ON T1.DATASOURCE_NUM_ID = I.DATASOURCE_NUM_ID
  AND T1.INTEGRATION_ID = I.INTEGRATION_ID
WHERE T1.SCD1_WID IS NULL

In [ ]:
%sql
-- Step 4: Insert dimension records with SCD1_WID and effective dates
INSERT INTO PRXBI_DW.W_GL_ACCOUNT_D
(
  ROW_WID,
  CHART_OF_ACCOUNTS,
  CONCATENATED_SEGMENT_VALUE,
  BALANCING_SEGMENT_NUM,
  BALANCING_SEGMENT_ATTRIB,
  COST_CENTER_NUM,
  COST_CENTER_ATTRIB,
  NATURAL_ACCOUNT_NUM,
  NATURAL_ACCOUNT_ATTRIB,
  RECON_TYPE_CODE,
  ACTIVE_FLG,
  CREATED_BY_ID,
  CHANGED_BY_ID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  AUX2_CHANGED_ON_DT,
  AUX3_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  EFFECTIVE_FROM_DT,
  EFFECTIVE_TO_DT,
  CURRENT_FLG,
  DELETE_FLG,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  TENANT_ID,
  X_CUSTOM,
  SCD1_WID,
  ETL_PROC_WID
)
SELECT
  row_number() OVER (ORDER BY S.INTEGRATION_ID) + 
  COALESCE((SELECT MAX(ROW_WID) FROM PRXBI_DW.W_GL_ACCOUNT_D), 0) AS ROW_WID,
  S.CHART_OF_ACCOUNTS,
  S.CONCATENATED_SEGMENT_VALUE,
  S.BALANCING_SEGMENT_NUM,
  S.BALANCING_SEGMENT_ATTRIB,
  S.COST_CENTER_NUM,
  S.COST_CENTER_ATTRIB,
  S.NATURAL_ACCOUNT_NUM,
  S.NATURAL_ACCOUNT_ATTRIB,
  S.RECON_TYPE_CODE,
  S.ACTIVE_FLG,
  S.CREATED_BY_ID,
  S.CHANGED_BY_ID,
  CAST(S.CREATED_ON_DT AS TIMESTAMP) AS CREATED_ON_DT,
  CAST(S.CHANGED_ON_DT AS TIMESTAMP) AS CHANGED_ON_DT,
  CAST(S.AUX1_CHANGED_ON_DT AS TIMESTAMP) AS AUX1_CHANGED_ON_DT,
  CAST(S.AUX2_CHANGED_ON_DT AS TIMESTAMP) AS AUX2_CHANGED_ON_DT,
  CAST(S.AUX3_CHANGED_ON_DT AS TIMESTAMP) AS AUX3_CHANGED_ON_DT,
  S.SRC_EFF_FROM_DT,
  S.EFFECTIVE_FROM_DT,
  S.EFFECTIVE_TO_DT,
  'Y' AS CURRENT_FLG,
  S.DELETE_FLG,
  S.DATASOURCE_NUM_ID,
  S.INTEGRATION_ID,
  S.TENANT_ID,
  S.X_CUSTOM,
  SCD1.SCD1_WID,
  2625250 AS ETL_PROC_WID
FROM I$_2625250_1 S
INNER JOIN PRXBI_DW.W_GL_ACCOUNT_T1_D SCD1
  ON SCD1.DATASOURCE_NUM_ID = S.DATASOURCE_NUM_ID
  AND SCD1.INTEGRATION_ID = S.INTEGRATION_ID
WHERE S.IND_UPDATE IN ('I', 'S')

In [ ]:
%sql
-- Step 5: Fix effective dates for historized records
MERGE INTO PRXBI_DW.W_GL_ACCOUNT_D AS T
USING (
  SELECT
    T0.ROW_WID,
    COALESCE(MIN(T1.EFFECTIVE_FROM_DT), to_date('3714-01-01', 'yyyy-MM-dd')) AS NEW_EFFECTIVE_TO_DT,
    CASE WHEN MIN(T1.EFFECTIVE_FROM_DT) IS NULL THEN 'Y' ELSE 'N' END AS NEW_CURRENT_FLG
  FROM PRXBI_DW.W_GL_ACCOUNT_D T0
  INNER JOIN PRXBI_DW.W_GL_ACCOUNT_D T1
    ON T0.DATASOURCE_NUM_ID = T1.DATASOURCE_NUM_ID
    AND T0.INTEGRATION_ID = T1.INTEGRATION_ID
    AND T0.EFFECTIVE_FROM_DT < T1.EFFECTIVE_FROM_DT
  WHERE T1.ETL_PROC_WID = 2625250
  GROUP BY T0.ROW_WID
) SRC
ON T.ROW_WID = SRC.ROW_WID
WHEN MATCHED THEN
  UPDATE SET
    T.EFFECTIVE_TO_DT = SRC.NEW_EFFECTIVE_TO_DT,
    T.CURRENT_FLG = SRC.NEW_CURRENT_FLG

In [ ]:
%sql
-- Step 6: Update reference dates in W_ETL_LOAD_DATES
MERGE INTO PRXBI_DW.W_ETL_LOAD_DATES AS TGT
USING (
  SELECT
    380 AS DATASOURCE_NUM_ID,
    'SILOS_SIL_GLACCOUNTDIMENSION' AS PACKAGE_NAME,
    'W_GL_ACCOUNT_D' AS TARGET_TABLE_NAME,
    '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
    2625250 AS ETL_PROC_WID,
    12345 AS LOAD_PLAN_ID,
    date_add(current_timestamp(), -30) AS WIP_LOAD_START_DATE,
    current_timestamp() AS ETL_LOAD_DATE,
    (SELECT CASE WHEN IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM VAR_IS_INCREMENTAL) AS COMMITTED
) SRC
ON TGT.DATASOURCE_NUM_ID = SRC.DATASOURCE_NUM_ID
  AND TGT.PACKAGE_NAME = SRC.PACKAGE_NAME
  AND TGT.ETL_USAGE_CODE = SRC.ETL_USAGE_CODE
WHEN MATCHED THEN
  UPDATE SET
    TGT.TARGET_TABLE_NAME = SRC.TARGET_TABLE_NAME,
    TGT.ETL_PROC_WID = SRC.ETL_PROC_WID,
    TGT.LOAD_PLAN_ID = SRC.LOAD_PLAN_ID,
    TGT.WIP_LOAD_START_DATE = SRC.WIP_LOAD_START_DATE,
    TGT.ETL_LOAD_DATE = SRC.ETL_LOAD_DATE,
    TGT.COMMITTED = SRC.COMMITTED
WHEN NOT MATCHED THEN
  INSERT (
    DATASOURCE_NUM_ID,
    PACKAGE_NAME,
    TARGET_TABLE_NAME,
    ETL_USAGE_CODE,
    ETL_PROC_WID,
    LOAD_PLAN_ID,
    WIP_LOAD_START_DATE,
    ETL_LOAD_DATE,
    COMMITTED
  )
  VALUES (
    SRC.DATASOURCE_NUM_ID,
    SRC.PACKAGE_NAME,
    SRC.TARGET_TABLE_NAME,
    SRC.ETL_USAGE_CODE,
    SRC.ETL_PROC_WID,
    SRC.LOAD_PLAN_ID,
    SRC.WIP_LOAD_START_DATE,
    SRC.ETL_LOAD_DATE,
    SRC.COMMITTED
  )

In [ ]:
%sql
-- Step 7: Insert reference dates to History Table W_ETL_LOAD_DATES_LOG
INSERT INTO PRXBI_DW.W_ETL_LOAD_DATES_LOG
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  2625250 AS SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
FROM PRXBI_DW.W_ETL_LOAD_DATES
WHERE DATASOURCE_NUM_ID = 380
  AND PACKAGE_NAME = 'SILOS_SIL_GLACCOUNTDIMENSION'
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 8: Drop work table (temp view)
DROP VIEW IF EXISTS I$_2625250_1;
DROP VIEW IF EXISTS VAR_IS_INCREMENTAL